<a href="https://colab.research.google.com/github/dohyun1411/Quora-Insincere-Questions-Classification/blob/preprocessing1/new_embedding_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

references:

https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go

https://www.kaggle.com/alhalimi/tokenization-and-word-embedding-compatibility

https://www.kaggle.com/canming/ensemble-mean-iii-64-36

https://www.kaggle.com/tks0123456789/pme-ema-6-x-8-epochs

https://www.kaggle.com/bkkaggle/pytorch-determinism-test

Please verify your config values.

In [71]:
seed = 31
n_splits = 5
input_path = '/content/drive/MyDrive/ColabNotebooks/QIQC/data'

In [72]:
def seed_torch(seed=seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [73]:
import gc

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [74]:
import os

train_path = os.path.join(input_path, 'train.csv')
test_path = os.path.join(input_path, 'test.csv')
embeddings_path = os.path.join(input_path, 'embeddings.zip')

In [75]:
if os.path.exists(train_path): train_df = pd.read_csv(train_path)
else: train_df = pd.read_csv(train_path + '.zip')

if os.path.exists(test_path): test_df = pd.read_csv(train_path)
else: test_df = pd.read_csv(test_path + '.zip')

print('train shape:', train_df.shape)
print('test shape:', test_df.shape)

train shape: (1306122, 3)
test shape: (1306122, 3)


In [4]:
import zipfile
from gensim.models import KeyedVectors

glove = 'glove.840B.300d/glove.840B.300d.txt'
wiki = 'wiki-news-300d-1M/wiki-news-300d-1M.vec'
google = 'GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
para = 'paragram_300_sl999/paragram_300_sl999.txt'


def load_embedding(embedding_name):

    def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')

    with zipfile.ZipFile(embeddings_path) as embeddings_zip:
        print("Found embeddings as a zip file")

        if embedding_name == google:
            return KeyedVectors.load_word2vec_format(embeddings_zip.open(google), binary=True)

        else:
            embedding = []
            for o in embeddings_zip.open(embedding_name):
                try:
                    if len(o.decode('utf-8')) > 100:
                        embedding.append(get_coefs(*o.decode('utf-8').split(" ")))
                except:
                    pass
        
        return dict(embedding)

In [5]:
%%time
glove_embedding = load_embedding(glove)

Found embeddings as a zip file
CPU times: user 3min 34s, sys: 4.11 s, total: 3min 38s
Wall time: 3min 39s


Some features about glove_embedding

In [6]:
"'re" in glove_embedding

True

In [7]:
"you're" in glove_embedding

True

In [8]:
"12345" in glove_embedding

True

In [10]:
"'m" in glove_embedding

True

In [242]:
for punct in '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~‘’“”…' + "'":
    if punct not in glove_embedding: print(punct)

We use all tokens from both train and test data for our vocabulary.

Tokenizer : keras

In [272]:
puncts = '´‘’“”…!#$%&()*+,-./:;<=>?@[\]^_`{|}~"' + "'"

In [273]:
from keras.preprocessing.text import Tokenizer

to_exclude = ''
to_tokenize = puncts

tokenizer = Tokenizer(filters=to_exclude, lower=False)

In [274]:
from sklearn.model_selection import train_test_split, KFold

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=seed)

train_df, val_df = train_test_split(train_df, test_size=0.1)

train_idx, val_idx = list(kfold.split(train_df))[0]
train_df, val_df = train_df.iloc[train_idx], train_df.iloc[val_idx]

train_text = train_df["question_text"].fillna("_na_")
val_text = val_df["question_text"].fillna("_na_")
test_text = test_df["question_text"].fillna("_na_")

train_y = train_df['target'].values
val_y = val_df['target'].values

In [275]:
import re

train_text = train_text.progress_apply(lambda x: re.sub(r'(['+to_tokenize+'])', r' \1', x)).values
val_text = val_text.progress_apply(lambda x: re.sub(r'(['+to_tokenize+'])', r' \1', x)).values
test_text = test_text.progress_apply(lambda x: re.sub(r'(['+to_tokenize+'])', r' \1', x)).values

In [276]:
text = list(train_text) + list(val_text) + list(test_text)

In [277]:
%%time
tokenizer.fit_on_texts(text)

CPU times: user 19.2 s, sys: 60.4 ms, total: 19.3 s
Wall time: 19.2 s


In [278]:
word_index = tokenizer.word_index

In [279]:
%%time
train_X = tokenizer.texts_to_sequences(train_text)
val_X = tokenizer.texts_to_sequences(val_text)
test_X = tokenizer.texts_to_sequences(test_text)

maxlen = max({len(seq) for seq in train_X} | {len(seq) for seq in val_X} | {len(seq) for seq in test_X})

CPU times: user 15.5 s, sys: 180 ms, total: 15.7 s
Wall time: 15.7 s


In [280]:
from keras.preprocessing.sequence import pad_sequences

train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

In [281]:
glove_emb_mean, glove_emb_std = -0.005838499, 0.48782197

In [282]:
embed_size = 300 # how big is each word vector
max_features = len(word_index) # how many unique words to use (i.e num rows in embedding vector)

Without any effort

In [283]:
show_inital_emb_matrix = False # set False if you do not want to see the intial embedding matrix

In [284]:
if show_inital_emb_matrix:
    glove_emb_matrix = np.random.normal(glove_emb_mean, glove_emb_std, (max_features, embed_size))
    glove_oov = {}
    for word, i in tqdm(word_index.items()):
        if word in glove_embedding:
            glove_emb_matrix[i] = glove_embedding[word]
        
        else:
            glove_oov[word] = id

    print('percentage of oov of glove: {:.2f}%'.format(len(glove_oov) / max_features * 100))

else:
    print('percentage of oov of glove: 41.39%')

percentage of oov of glove: 41.39%


Let's make an improved embedding matrix.

In [285]:
import nltk
# nltk.download('wordnet')

from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
s = PorterStemmer()
l = LancasterStemmer()
n = WordNetLemmatizer()

In [286]:
def is_in_emb(word, i, embedding, emb_matrix):

    if word in embedding:
        emb_matrix[i] = embedding[word]
        return True
    
    tmp = word.lower()
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    tmp = word.upper()
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True

    tmp = word.capitalize()
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True

    tmp = s.stem(word)
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    tmp = l.stem(word)
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    tmp = n.lemmatize(word)
    if tmp in embedding:
        emb_matrix[i] = embedding[tmp]
        return True
    
    return False

In [287]:
glove_emb_matrix = np.random.normal(glove_emb_mean, glove_emb_std, (max_features, embed_size))
glove_oov = {}
# double_continue = False
for word, i in tqdm(word_index.items()):
    i -= 1
    assert i >= 0

    if is_in_emb(word, i, glove_embedding, glove_emb_matrix): continue

    tmp = word
    for punct in puncts:
        tmp = tmp.replace(punct, '')
    if is_in_emb(tmp, i, glove_embedding, glove_emb_matrix): continue

    for num in '0123456789':
        tmp = tmp.replace(num, '')
    if is_in_emb(tmp, i, glove_embedding, glove_emb_matrix): continue




    



    else:
        glove_oov[word] = i

print('percentage of oov of glove: {:.2f}%'.format(len(glove_oov) / max_features * 100))


percentage of oov of glove: 14.11%


In [ ]:
glove_oov

41.39% -> 14.11%

Wiki